In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="model_float16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
alphabet = "0123456789abcdefghijklmnopqrstuvwxyz."
blank_index = len(alphabet)

c:\Users\91955\anaconda3\envs\.weighout\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [3]:
# Open camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Crop / select ROI if needed
    roi = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (200,31)).astype('float32')/255
    inp = np.expand_dims(roi, axis=(0,-1))  # shape [1,31,200,1]
    
    # Run inference
    interpreter.set_tensor(input_details[0]['index'], inp)
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]['index'])
    
    # Decode
    output_array = result[0]
    pred_indices = np.argmax(output_array, axis=-1) if output_array.ndim==2 else output_array.astype(int)
    text = "".join(alphabet[i] for i in pred_indices if i not in [blank_index])
    
    # Overlay on frame
    cv2.putText(frame, text, (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.imshow("7-Segment OCR", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()